# Camera calibration

In [ ]:
import os
import glob
import cv2
import copy
import numpy as np
from utils import *
from utils.frame_extractor import frame_extractor

### Create calibration images

In [ ]:
data_folder = "../data/calibration"
frame_extractor(camera_index=0, width=1280, height=720, output_dir=data_folder)

#### Load images

In [ ]:
imgs_path = glob.glob(os.path.join(data_folder, '*.jpg'))
imgs = [cv2.imread(img_path) for img_path in imgs_path]

#### Detect and refine corners

In [73]:
corners = [cv2.findChessboardCorners(imgs[i], (8,6)) for i in range(len(imgs))]
for i, foto in enumerate(corners):
    if not foto[0]:
        print(f"Corners on image {imgs_path[i]} couldn't be found")
corners_copy = copy.deepcopy(corners)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 27, 0.01)

imgs_gray = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in imgs]

corners_refined = [cv2.cornerSubPix(i, cor[1], (8, 6), (-1, -1), criteria) if cor[0] else [] for i, cor in zip(imgs_gray, corners_copy)]

In [ ]:
imgs_copy = copy.deepcopy(imgs)
corners_drawn_imgs = [cv2.drawChessboardCorners(img_copy, (8,6), corner_r, True) for img_copy, corner, corner_r in zip(imgs_copy, corners, corners_refined) if corner[0]]
output_dir = "../results/calibration"
os.makedirs(output_dir, exist_ok=True)

for i,image in enumerate(corners_drawn_imgs):
    img_name = f"{i}"
    u.show_image(image, img_name)
    u.write_image(output_dir, img_name, image)

#### Obtain parameters

In [75]:
def get_chessboard_points(chessboard_shape, dx, dy):
    puntos = []
    for j in range(chessboard_shape[1]):
        for i in range(chessboard_shape[0]):
            puntos.append([dx*i, dy*j, 0])
    return np.array(puntos, np.float32)

In [76]:
chessboard_points = get_chessboard_points((8, 6), 27, 27)

In [77]:
valid_corners = [cor[1] for cor in corners if cor[0]]
# Convert list to numpy array
valid_corners = np.asarray(valid_corners, dtype=np.float32)
print(imgs_copy[1].shape)
obj_points = [chessboard_points for _ in range(len(valid_corners))]
rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(obj_points, valid_corners, (1280, 720), None, None)

# Obtain extrinsics
extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))
print("Intrinsics:\n", intrinsics)
print("Extrinsics:\n", extrinsics)
# Para la imagen 1 la matriz extrinseca es extrinsics[0]
print("Distortion coefficients:\n", dist_coeffs)
print("Root mean squared reprojection error:\n", rms)

(720, 1280, 3)
Intrinsics:
 [[968.7907546    0.         630.73825542]
 [  0.         966.89881871 376.66333399]
 [  0.           0.           1.        ]]
Extrinsics:
 [array([[ 9.88158459e-01, -6.90212168e-02, -1.37036244e-01,
        -2.56532868e+01],
       [ 7.57634232e-02,  9.96127710e-01,  4.46036800e-02,
        -7.19616389e+01],
       [ 1.33427000e-01, -5.44578386e-02,  9.89561307e-01,
         4.17175125e+02]]), array([[ 9.87564222e-01,  1.54769059e-01,  2.76305308e-02,
        -1.26225620e+02],
       [-1.53949857e-01,  9.16356668e-01,  3.69580706e-01,
        -5.51693012e+01],
       [ 3.18802371e-02, -3.69238399e-01,  9.28787734e-01,
         4.09125841e+02]]), array([[ 9.82303768e-01,  1.85564410e-01, -2.53999265e-02,
        -1.42781515e+02],
       [-1.84239957e-01,  9.81748885e-01,  4.71674070e-02,
        -5.20031946e+01],
       [ 3.36889416e-02, -4.16530403e-02,  9.98564009e-01,
         3.49027826e+02]]), array([[ 9.85471405e-01,  1.60631923e-01, -5.51678803e-02,
 